# Assignment 4

Before working on this assignment please read these instructions fully. In the submission area, you will notice that you can click the link to **Preview the Grading** for each step of the assignment. This is the criteria that will be used for peer grading. Please familiarize yourself with the criteria before beginning the assignment.

This assignment requires that you to find **at least** two datasets on the web which are related, and that you visualize these datasets to answer a question with the broad topic of **economic activity or measures** (see below) for the region of **Ann Arbor, Michigan, United States**, or **United States** more broadly.

You can merge these datasets with data from different regions if you like! For instance, you might want to compare **Ann Arbor, Michigan, United States** to Ann Arbor, USA. In that case at least one source file must be about **Ann Arbor, Michigan, United States**.

You are welcome to choose datasets at your discretion, but keep in mind **they will be shared with your peers**, so choose appropriate datasets. Sensitive, confidential, illicit, and proprietary materials are not good choices for datasets for this assignment. You are welcome to upload datasets of your own as well, and link to them using a third party repository such as github, bitbucket, pastebin, etc. Please be aware of the Coursera terms of service with respect to intellectual property.

Also, you are welcome to preserve data in its original language, but for the purposes of grading you should provide english translations. You are welcome to provide multiple visuals in different languages if you would like!

As this assignment is for the whole course, you must incorporate principles discussed in the first week, such as having as high data-ink ratio (Tufte) and aligning with Cairo’s principles of truth, beauty, function, and insight.

Here are the assignment instructions:

 * State the region and the domain category that your data sets are about (e.g., **Ann Arbor, Michigan, United States** and **economic activity or measures**).
 * You must state a question about the domain category and region that you identified as being interesting.
 * You must provide at least two links to available datasets. These could be links to files such as CSV or Excel files, or links to websites which might have data in tabular form, such as Wikipedia pages.
 * You must upload an image which addresses the research question you stated. In addition to addressing the question, this visual should follow Cairo's principles of truthfulness, functionality, beauty, and insightfulness.
 * You must contribute a short (1-2 paragraph) written justification of how your visualization addresses your stated research question.

What do we mean by **economic activity or measures**?  For this category you might look at the inputs or outputs to the given economy, or major changes in the economy compared to other regions.

## Tips
* Wikipedia is an excellent source of data, and I strongly encourage you to explore it for new data sources.
* Many governments run open data initiatives at the city, region, and country levels, and these are wonderful resources for localized data sources.
* Several international agencies, such as the [United Nations](http://data.un.org/), the [World Bank](http://data.worldbank.org/), the [Global Open Data Index](http://index.okfn.org/place/) are other great places to look for data.
* This assignment requires you to convert and clean datafiles. Check out the discussion forums for tips on how to do this from various sources, and share your successes with your fellow students!

## Example
Looking for an example? Here's what our course assistant put together for the **Ann Arbor, MI, USA** area using **sports and athletics** as the topic. [Example Solution File](./readonly/Assignment4_example.pdf)

#### Importing required files :

In [ ]:
import sqlite3
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

%matplotlib notebook

#### Creating a connection to the sqlite database :

In [ ]:
conn = sqlite3.connect("./../../../../Datasets/European soccer databse 2008-16/database.sqlite")

#### Fetching required tables form the dataset and storing them in the required database :

In [ ]:
countries = pd.read_sql_query("select * from Country", conn)
leagues =  pd.read_sql_query("Select * from League", conn)
matches = pd.read_sql_query("Select * from Match", conn)
teams = pd.read_sql_query("Select * from Team", conn)
team_attributes = pd.read_sql_query("Select * from Team_Attributes", conn)
# players = pd.read_sql_query("Select * from Player", conn)
# player_attributes = pd.read_sql_query("Select * from Player_Attributes", conn)

### Understanding the structure of dataframes :

#### Printing top 5 columns to understand the structure and identify required attributes and data frames :

In [ ]:
print("COUNTRIES: \n \n",countries.head(),
      "\n \n LEAGUES: \n \n",leagues.head(),
      "\n \n TEAMS: \n \n",teams.head(),
      "\n \n PLAYERS: \n \n",players.head()
     )

In [ ]:
matches.head()

###   _In this notebook i will concentrate on the performances of the top teams in the English Premier League :_ 

#### Reducing the Matches dataset to only specific league :

In [ ]:
# i/p : league_name ; o/p : league_id (id of the specied league)
def get_league_id(league_name) :
    league_id = leagues[leagues["name"] == "England Premier League"].loc[:,"id"].iloc[0]
    return league_id

# i/p : league_id; o/p : league_matches_dataframe - ( all the matches from a specific league )
def get_league_matches(league_id):
    df = matches[matches["league_id"] == league_id]
    return df

In [ ]:
# fetching all EPL matches from the matches dataframe
league_name = "England Premier League"
league_id = get_league_id(league_name)

# EPL matches dataframe
epl_matches_df = get_league_matches(league_id) 
print("EPL matches dataframe structure : \n \n Rows = {} Columns = {} \n \n Top 5 columns : \n \n {}"
      .format(epl_matches_df.shape[0],
              epl_matches_df.shape[1],
              epl_matches_df.head())
     )

#### Removing unncessary columns from EPL matches dataframe:

In [ ]:
# REMOVING : PLAYER INFORMATION COLUMNS, MATCH EVENT COLUMNS AND BETTING ODDS COLUMNS FOR THAT MATCH:
# REMOVING: country Id, league Id and date of the match information
# Setting index to 'id column'
# renaming some columns {'home_team_api_id' : 'home_team_id', 'away_team_api_id': 'away_team_id','stage' : 'match_day'}
epl_matches_df = (epl_matches_df.iloc[:,0:11]
                  .drop(columns=['country_id', 'league_id', 'date'])
                  .set_index('id')
                  .rename({'home_team_api_id' : 'home_team_id', 'away_team_api_id': 'away_team_id','stage' : 'match_day', 'match_api_id' : 'match_id'}, axis= 1)
                 )


In [ ]:
# i/p : list of team ids o/p : dict of team id and team name
def get_team_name(team_id_list):
    team_long_name = {}
    team_short_name = {}
    for id in team_id_list:
        team_long_name[id] = teams[teams['team_api_id'] == id]['team_long_name'].iloc[0]
        team_short_name[id] = teams[teams['team_api_id'] == id]['team_short_name'].iloc[0]
    return (team_long_name, team_short_name)

In [ ]:
def add_result_column(row):
    if row['home_team_goal'] == row['away_team_goal']:
        row["result"] = 'draw'
    elif row['home_team_goal'] > row['away_team_goal']:
        row["result"] = 'home'
    else:
        row['result'] = 'away'
    return row    

def add_team_names(row, team_name_dict):
    row["home_team"] = team_name_dict[row['home_team_id']]
    row["away_team"] = team_name_dict[row['away_team_id']]
    return row

full_name_dict, short_name_dict = get_team_name(epl_matches_df['home_team_id'].unique())

# adding result column
epl_matches_df = (epl_matches_df.apply(add_result_column, axis=1)
                                .apply(add_team_names, args=(full_name_dict,), axis=1))

In [ ]:
epl_matches_df.head()

#### Finding the top 4 performing teams in the premier league from 2008-16 :
 - Top performing teams over a period is calculated based on **win percentage** of a team over that period.
 - In case of same win percentage we consider **draw %** 

In [ ]:
def set_winner(row):
    if row['result'] == 'home':
        row['winner'] = row['home_team']
    elif row['result'] == 'away':
        row['winner'] = row['away_team']
    else:
        row['winner'] = 'draw'
    return row

epl_matches_df = epl_matches_df.apply(set_winner, axis = 1)

In [ ]:
top_6_teams = (epl_matches_df.groupby('winner')['match_id']
                       .agg({'count'})
                       .rename({'count' : 'win_count'}, axis = 1)
                       .sort_values('win_count',axis = 0, ascending = False)
                       .iloc[1:7,:]
        )

# top teams over the specific period based on win count
top_6_teams

#### Reducing the matches dataset to keep only the top 6 teams required :

In [ ]:
epl_matches_top_6 = epl_matches_df[epl_matches_df['home_team'].isin(top_6_teams.index.tolist()) | epl_matches_df['away_team'].isin(top_6_teams.index.tolist())]

In [ ]:
epl_matches_top_6.groupby('season')